修改所有的 ```pin_memory``` 以降低記憶體爆掉的問題。
```diff
- valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
+ valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=False)
```


**先做做看 Hard 的要求**，主要參考 HW03.pdf 的最後一頁 Resource 中的提示跟教授的影片。

- https://youtu.be/fX_guE7JNnY?t=1163 中提到，怎麼選資料是 open question


先嘗試 Entropy-based Regularization

# Entropy-based Regularization

參考 [影片時間](https://youtu.be/fX_guE7JNnY?t=1656) 跟 [Entropy Regularization in Reinforcement Learning](https://towardsdatascience.com/entropy-regularization-in-reinforcement-learning-a6fa6d7598df)

公式：
$$
H(X) = -\sum\pi\left(x\right)\log\left(\pi\left(x\right)\right) 
$$

若是越集中在某一個 class(以機率或數值表示)，entropy 數值就會接近零。若是平均分佈，數值就會遠離零。對應 python code(下個 Cell)，第三個例子的數值有 0.8 但是離零有段距離，所以 ```get_pseudo_labels``` 的參數 ```threshold``` 要調整。

In [4]:
import numpy as np
q_1 = [0.26, 0.28, 0.24, 0.22]
h_1 = -np.sum(q_1*np.log(q_1))
print(h_1)

q_1 = [0.26, 0.28, 0.24, 0.22] * 4
h_1 = -np.sum(q_1*np.log(q_1))
print(h_1)

q_1 = [0.1, 0.08, 0.8, 0.02]
h_1 = -np.sum(q_1*np.log(q_1))
print(h_1)

q_1 = [0.5, 0.08, 0.5, 0.02]
h_1 = -np.sum(q_1*np.log(q_1))
print(h_1)

q_1 = [0.6, 0.08, 0.4, 0.02]
h_1 = -np.sum(q_1*np.log(q_1))

print(h_1)

1.3822855642311125
5.52914225692445
0.6890721020039956
0.9734459322131686
(array([0, 4]),)
-18.821710777363496


In [11]:
a = np.array([1,2,3,4,5])
b = a > 3
c = np.where(b)[0]
print(c)
print(type(c))

[3 4]
<class 'numpy.ndarray'>


## 取出部份資料
這裡遇到了一個難題，我要怎麼取出部份資料，並且為 DataSetFolder 型態。
在 github 搜尋到 [@1am9trash](https://github.com/1am9trash/Hung_Yi_Lee_ML_2021/blob/main/hw/hw3/hw3_code.ipynb) 撰寫的程式碼，擷取實作程式碼。

```python
class PseudoDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, id):
        return self.x[id][0], self.y[id]

......

    idx = []
    labels = []

    for i, batch in enumerate(data_loader):
        img, _ = batch
        with torch.no_grad():
            logits = model(img.to(device))
        probs = softmax(logits)

        for j, x in enumerate(probs):
            if torch.max(x) > threshold:
                idx.append(i * batch_size + j)
                labels.append(int(torch.argmax(x)))
    
    dataset = PseudoDataset(Subset(dataset, idx), labels)
```

最後一段得知可以使用 [Subset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset)，定睛一看發現作業本身就有寫了
```python
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
```



第一個寫法會產生奇怪的問題，改成類似於 @1am9trash 的寫法。

第一次的程式碼：
```python
class PseudogDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        ()

    def __getitem__(self, index):
        return self.x[index][0], self.y[index]

    def __len__(self):
        return len(self.y)
```

```python
    selected_indices = []
    selected_labels = []

    # Iterate over the dataset by batches.
    for i, batch in enumerate(data_loader):

        img, _ = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)

        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        entropy_value = -torch.sum(probs*torch.log(probs), -1)
        selected = entropy_value < threshold
        selected[0] = True # 測試用
        selected[1] = True # 測試用

        batch_selected = torch.add(torch.where(selected)[0], i * batch_size)
        selected_indices.append(batch_selected)
        batch_label = torch.argmax(probs[selected], -1)
        selected_labels.append(batch_label)

    selected_indices = torch.cat(selected_indices)
    selected_labels = torch.cat(selected_labels).int()
    
    dataset = PseudogDataset(Subset(dataset, selected_indices), selected_labels)
```

改成以下程式碼：

```python
    entropy_value = -torch.sum(probs*torch.log(probs), -1)
            argmax = torch.argmax(probs, -1)
            for j, prob in enumerate(probs):
                if entropy_value[j] < threshold:
                # if j == 0 or j == 1:
                    selected_indices.append(i * batch_size + j)
                    selected_labels.append(int(torch.argmax(prob)))

dataset = PseudogDataset(Subset(dataset, selected_indices), selected_labels)
```

# 第一次改進

對比 [@1am9trash](https://github.com/1am9trash/Hung_Yi_Lee_ML_2021/blob/main/hw/hw3/hw3_code.ipynb) 撰寫的程式碼中的提示，要先思考 Easy, Medium 的作業要求。